# Model training

In [1]:
import torch
from torch import nn
import pandas as pd

In [2]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# Load data

X_train = pd.read_csv("./../data/features/in_features_train.csv")
# X_train = torch.from_numpy(X_train.to_numpy()).type(torch.float)

X_train_pca = pd.read_csv("./../data/features/pca_in_features_train.csv")
# X_train_pca = torch.from_numpy(X_train_pca.to_numpy()).type(torch.float)
X_train.shape, X_train_pca.shape

((62, 2552), (62, 61))

In [4]:
y_train = pd.read_csv("./../data/features/out_features_train.csv").label
# y_train = torch.from_numpy(y_train.label.to_numpy()).type(torch.LongTensor)
y_train.shape

(62,)

In [5]:
X_test = pd.read_csv("./../data/features/in_features_test.csv")
# X_test = torch.from_numpy(X_test.to_numpy()).type(torch.float)

X_test_pca = pd.read_csv("./../data/features/pca_in_features_test.csv")
# X_test_pca = torch.from_numpy(X_test_pca.to_numpy()).type(torch.float)
X_test.shape, X_test_pca.shape

((16, 2552), (16, 61))

In [6]:
y_test = pd.read_csv("./../data/features/out_features_test.csv").label
# y_test = torch.from_numpy(y_test.label.to_numpy()).type(torch.LongTensor)
y_test.shape

(16,)

## Support Vector Machine (SVM)

### Non PCA features

In [7]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, KFold

svm = SVC(kernel='rbf')
p_grid = {"C": [1, 10, 100], "gamma": [0.01, 0.1]}
cv = KFold(n_splits=5, shuffle=True)

clf = GridSearchCV(estimator=svm, param_grid=p_grid, cv=cv, scoring="balanced_accuracy")
clf.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [1, 10, 100], 'gamma': [0.01, 0.1]},
             scoring='balanced_accuracy')

In [8]:
clf.predict(X_test)

array([0, 0, 0, 2, 1, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0], dtype=int64)

In [9]:
y_test.to_numpy()

array([0, 0, 2, 3, 3, 0, 1, 3, 2, 2, 0, 2, 0, 0, 0, 1], dtype=int64)

In [10]:
# Evaluate

print("SVM Balanced Accuracy: ", clf.score(X_test, y_test))

SVM Balanced Accuracy:  0.2767857142857143


In [11]:
import joblib
from pathlib import Path


# Create models directory (if it doesn't already exist), see: https://docs.python.org/3/library/pathlib.html#pathlib.Path.mkdir
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, # create parent directories if needed
                 exist_ok=True # if models directory already exists, don't error
)

# Create model save path
MODEL_NAME = "svm.pk"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

joblib.dump(clf, MODEL_SAVE_PATH)

['models\\svm.pk']

### PCA features


In [12]:
svm = SVC(kernel='rbf')
p_grid = {"C": [1, 10, 100], "gamma": [0.01, 0.1]}
cv = KFold(n_splits=5, shuffle=True)

clf_pca = GridSearchCV(estimator=svm, param_grid=p_grid, cv=cv, scoring="balanced_accuracy")
clf_pca.fit(X_train_pca, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             estimator=SVC(),
             param_grid={'C': [1, 10, 100], 'gamma': [0.01, 0.1]},
             scoring='balanced_accuracy')

In [13]:
clf_pca.predict(X_test_pca)

array([0, 0, 0, 2, 1, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0], dtype=int64)

In [14]:
y_test.to_numpy()

array([0, 0, 2, 3, 3, 0, 1, 3, 2, 2, 0, 2, 0, 0, 0, 1], dtype=int64)

In [15]:
# Evaluate

print("SVM Balanced Accuracy: ", clf_pca.score(X_test_pca, y_test))

SVM Balanced Accuracy:  0.2767857142857143


## Linear model

## Non-PCA

In [16]:
X_train = torch.from_numpy(X_train.to_numpy()).type(torch.float)
X_test = torch.from_numpy(X_test.to_numpy()).type(torch.float)

classes = y_train.unique()

y_train = torch.from_numpy(y_train.to_numpy()).type(torch.LongTensor)
y_test = torch.from_numpy(y_test.to_numpy()).type(torch.LongTensor)

in_features, out_features = X_train.shape[1], len(classes)
in_features, out_features

(2552, 5)

In [17]:
from modules.model_builder import BaseModel

model_0 = BaseModel(in_features, 10, out_features).to(device)

In [18]:
from modules.engine import train, save_model

loss_fn = nn.CrossEntropyLoss()
optimizer_class = torch.optim.SGD

train(model_0, X_train, X_test, y_train, y_test,
      loss_fn, optimizer_class, 1000, 0.1, device)

In [19]:
model_0.eval()
with torch.inference_mode():
    y_logits = model_0(X_test.to(device))
    y_preds = y_logits.argmax(dim=1).to("cpu")
y_preds

tensor([0, 0, 0, 2, 1, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2])

In [20]:
from sklearn.metrics import accuracy_score

print("MLP Accuracy: ", accuracy_score(y_test, y_preds))

MLP Accuracy:  0.4375


In [21]:
save_model(model_0, "mlp.pth")

Saving model to: models\mlp.pth


## PCA Model

In [22]:
X_train_pca = torch.from_numpy(X_train_pca.to_numpy()).type(torch.float)
X_test_pca = torch.from_numpy(X_test_pca.to_numpy()).type(torch.float)


In [23]:
in_features_pca = X_train_pca.shape[1]
in_features_pca, out_features

(61, 5)

In [24]:
model_0_pca = BaseModel(in_features_pca, 10, out_features).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer_class = torch.optim.SGD

train(model_0_pca, X_train_pca, X_test_pca, y_train, y_test,
      loss_fn, optimizer_class, 1000, 0.1, device)

In [25]:
model_0_pca.eval()
with torch.inference_mode():
    y_logits = model_0_pca(X_test_pca.to(device))
    y_preds = y_logits.argmax(dim=1).to("cpu")
y_preds

tensor([0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0])

In [26]:
print("MLP + PCA Accuracy: ", accuracy_score(y_test, y_preds))

MLP + PCA Accuracy:  0.5


In [27]:
save_model(model_0_pca, "mlp_pca.pth")

Saving model to: models\mlp_pca.pth


## Convolution Model

### Non-PCA

In [28]:
X_train = X_train.unsqueeze(1)
X_test = X_test.unsqueeze(1)

In [29]:
in_channels, in_length = X_train.shape[1:]
in_channels, in_length

(1, 2552)

In [30]:
from modules.model_builder import ConvModel

model_1 = ConvModel(in_channels, in_length, 10, out_features).to(device)

638


In [31]:
train(model_1, X_train, X_test, y_train, y_test,
      loss_fn, optimizer_class, 100, 0.1, device)

In [32]:
model_1.eval()
with torch.inference_mode():
    y_logits = model_1(X_test.to(device))
    y_preds = y_logits.argmax(dim=1).to("cpu")
y_preds

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [33]:
y_preds

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [34]:
print("CNN Accuracy: ", accuracy_score(y_test, y_preds))

CNN Accuracy:  0.4375


In [35]:
save_model(model_1, "cnn.pth")

Saving model to: models\cnn.pth


### PCA

In [36]:
X_train_pca = X_train_pca.unsqueeze(1)
X_test_pca = X_test_pca.unsqueeze(1)

In [37]:
in_channels, in_length = X_train_pca.shape[1:]
in_channels, in_length

(1, 61)

In [38]:
model_1_pca = ConvModel(in_channels, in_length, 10, out_features).to(device)

16


In [39]:
train(model_1_pca, X_train_pca, X_test_pca, y_train, y_test,
      loss_fn, optimizer_class, 100, 0.1, device)

In [40]:
model_1_pca.eval()
with torch.inference_mode():
    y_logits = model_1_pca(X_test_pca.to(device))
    y_preds = y_logits.argmax(dim=1).to("cpu")
y_preds

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [41]:
y_preds

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [42]:
print("CNN + PCA Accuracy: ", accuracy_score(y_test, y_preds))

CNN + PCA Accuracy:  0.4375


In [43]:
save_model(model_1, "cnn_pca.pth")

Saving model to: models\cnn_pca.pth
